In [1]:
import pandas as pd
import numpy as np
from itertools import chain
from collections import defaultdict

In [2]:
total_movies = pd.read_csv('./KOFIC/Korean Movies Info.csv')

In [3]:
total_movies['전국 관객수']

0       17613682
1       16264944
2       14410754
3       14245998
4       13395400
          ...   
2307        1008
2308        1006
2309        1006
2310        1005
2311        1000
Name: 전국 관객수, Length: 2312, dtype: int64

In [4]:
total_movies.loc[0, '전국 관객수']

17613682

In [5]:
total_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2312 entries, 0 to 2311
Data columns (total 23 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   순번        2312 non-null   int64 
 1   영화명       2312 non-null   object
 2   감독        2299 non-null   object
 3   제작사       2275 non-null   object
 4   수입사       2 non-null      object
 5   배급사       2301 non-null   object
 6   개봉일       2312 non-null   object
 7   영화유형      2312 non-null   object
 8   영화형태      2312 non-null   object
 9   국적        2312 non-null   object
 10  전국 스크린수   2312 non-null   object
 11  전국 매출액    2288 non-null   object
 12  전국 관객수    2312 non-null   int64 
 13  서울 매출액    2287 non-null   object
 14  서울 관객수    2312 non-null   object
 15  장르        2312 non-null   object
 16  등급        2312 non-null   object
 17  영화구분      2312 non-null   object
 18  주연 배우     2295 non-null   object
 19  주연 배우 코드  2295 non-null   object
 20  조연 배우     1966 non-null   object
 21  조연 배우 코드  1966

## 주연 배우 서브 데이터베이스 만들기
### 필요 column :
 - 배우 코드
 - 배우 이름
 - 찍은 영화 리스트
 - 찍은 영화 편수
 - 총 관람객
 - 평균 관람객
 - 최근 영화 5개
 - 최근 영화 5개의 평균 관람객

## 배우 서브 데이터베이스 생성

In [6]:
# total_movies['Key'] = total_movies['영화명'].fillna("") + " : " + total_movies['감독'].fillna("")

In [7]:
#unique한 배우 코드 가져오기

actor_code_list = []
actor_name_list = []

for codes_string, names_string in zip(total_movies['주연 배우 코드'].fillna(""), total_movies['주연 배우'].fillna("")):
    # print(codes_string, type(codes_string))
    codes = codes_string.split(", ")
    # print(names_string, type(names_string))
    names = names_string.split(", ")
    if(len(codes)!=len(names)):
        print(codes, names)
    for code, name in zip(codes, names):
        if(code not in actor_code_list):
            actor_code_list.append(code)
            actor_name_list.append(name)

In [8]:
print(len(actor_code_list),len(actor_name_list))

3981 3981


In [9]:
total_movies['개봉일date'] = pd.to_datetime(total_movies['개봉일'])

In [10]:
#찍은 영화 인덱스
actor_film_list = defaultdict(list)
#찍은 영화 편수
actor_film_num = defaultdict(int)
#총 관람객
actor_total_visitor = defaultdict(int)

#최근 영화 5개 인덱스
#(인덱스, 영)
actor_film_list_lately5 = defaultdict(list)

for index, codes_string in zip(total_movies.index, total_movies['주연 배우 코드'].fillna("")):
    codes = codes_string.split(", ")
    for code in codes:
        actor_film_list[code].append(index)
        actor_film_num[code] += 1
        actor_total_visitor[code] += total_movies.loc[index, '전국 관객수']
        
        date = total_movies.loc[index, '개봉일date']
        # print(code, index, date)
        if(len(actor_film_list_lately5[code])<1):
            actor_film_list_lately5[code].append((index, date))
        else:
            n = 0
            for lately5_index,lately5_date in actor_film_list_lately5[code]:
                if(date>lately5_date):
                    actor_film_list_lately5[code].insert(n, (index, date))
                    break
                n += 1
                if(n==5):
                    actor_film_list_lately5[code].append((index, date))
        if(len(actor_film_list_lately5[code])>5):
            actor_film_list_lately5[code].pop()


In [11]:
actor_df = pd.DataFrame(actor_name_list, index=actor_code_list, columns = ['배우명'])
actor_df

,배우명
10072251,최민식
10019065,류승룡
10067353,조진웅
20125825,이하늬
20137049,진선규
...,...
20363318,김여진
20120069,지용석
20183250,박지혜
20382491,김정주


In [12]:
tmp_dict = defaultdict(list)

col_dicts = [actor_film_list, actor_film_num, actor_total_visitor, actor_film_list_lately5]
col_dfs = [col.items() for col in col_dicts]

for k, v in chain(col_dfs[0], col_dfs[1], col_dfs[2], col_dfs[3]):
    # print(k)
    tmp_dict[k].append(v)

In [13]:
actor_info = pd.DataFrame.from_dict(tmp_dict, orient='index', columns=['촬영 영화', '촬영 영화 편수', '배우 총 관람객', '최근 촬영 영화'])

In [14]:
actor_info

,촬영 영화,촬영 영화 편수,배우 총 관람객,최근 촬영 영화
10072251,"[0, 84, 85, 117, 133, 222, 252, 277, 289, 295,...",15,46570842,"[(650, 2022-03-09 00:00:00), (252, 2019-12-26 ..."
10019065,"[0, 1, 7, 10, 34, 87, 158, 164, 270, 298, 459,...",25,86251013,"[(953, 2022-09-28 00:00:00), (658, 2021-11-17 ..."
10067353,"[0, 70, 75, 76, 82, 95, 124, 158, 183, 193, 20...",26,66667732,"[(575, 2022-01-05 00:00:00), (1483, 2021-11-11..."
20125825,"[1, 90, 104, 193, 337, 350, 542, 681, 938, 100...",11,31763579,"[(337, 2022-07-20 00:00:00), (193, 2019-11-13 ..."
20137049,"[1, 58, 113, 433, 450, 450, 1019, 1019, 1110, ...",10,29489745,"[(58, 2022-09-07 00:00:00), (1110, 2021-12-01 ..."
...,...,...,...,...
20363318,[2308],1,1006,"[(2308, 2020-03-19 00:00:00)]"
20120069,[2309],1,1006,"[(2309, 2018-07-19 00:00:00)]"
20183250,[2310],1,1005,"[(2310, 2014-06-26 00:00:00)]"
20382491,[2311],1,1000,"[(2311, 2022-05-12 00:00:00)]"


In [15]:
actor_film_list

defaultdict(list,
            {'10072251': [0,
              84,
              85,
              117,
              133,
              222,
              252,
              277,
              289,
              295,
              370,
              563,
              650,
              672,
              1547],
             '10019065': [0,
              1,
              7,
              10,
              34,
              87,
              158,
              164,
              270,
              298,
              459,
              473,
              529,
              622,
              654,
              658,
              764,
              791,
              838,
              944,
              953,
              1168,
              1208,
              1443,
              1468],
             '10067353': [0,
              70,
              75,
              76,
              82,
              95,
              124,
              158,
              183,
              193,
         

In [16]:
actor_file

NameError: name 'actor_file' is not defined

In [ ]:
actor_film_num

In [ ]:
actor_film_list

In [ ]:
actor_film_list_lately5

In [ ]:
for k, v in chain([(1,1), (1, 2)], [(1, 3)], [(2, 3)]):
    print(k, v)